In [23]:
import requests
from bs4 import BeautifulSoup
import jieba
import jieba.analyse
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.cluster import KMeans
from time import time
import numpy as np

In [24]:
# 获取交大新闻网主页新闻的链接列表
news_url_list = []

nav_page = 1654
while len(news_url_list)<100:
    nav_url = "http://news.xjtu.edu.cn/zyxw/"
    nav_url = nav_url + str(nav_page) + ".htm"
    r = requests.get(nav_url)
    if r.status_code != 200:
        raise Exception()
    r.encoding='utf-8'

    soup = BeautifulSoup(r.text, "html.parser")
    for node in soup.find_all("a"):
        if node.has_attr('href') and not node.attrs['href'].find(u'../info/')==-1:
            news_url_list.append(node.attrs['href'])
            # print(node.attrs['href'])
    news_url_list = list(set(news_url_list))
    nav_page = nav_page - 1

print(news_url_list[0])

../info/1033/129690.htm


In [25]:
# 获取新闻内容
news_list = []
for news_url in news_url_list:
    news_url = news_url.replace('../', 'http://news.xjtu.edu.cn/')
    # print(news_url)
    
    r = requests.get(news_url)
    if r.status_code != 200:
        raise Exception()
    r.encoding='utf-8'

    soup = BeautifulSoup(r.text, "html.parser")
    text = str()
    for node in soup.find_all("p"):
        if node.get_text().find(u'上一条')==-1 and node.get_text().find(u'下一条')==-1 and node.get_text().find(u'关闭')==-1 and node.get_text().find(u'版权所有')==-1 and node.get_text().find(u'来源链接')==-1 and len(node.get_text().strip(' '))>0:
            text = text + node.get_text().strip(' ')
    news_list.append(text) 
    # print(text)

print(news_list[0])

春到人间草木知，西安交大校园内杨柳依依、梅花浅笑，迎接着2021年春季学期返校的学子们。根据教育部、陕西省及西安市疫情防控要求，学校按照“分期分批、有序返校”的原则，安排国（境）内学生2月27日—28日返校注册。学校各部门齐上阵，安全隐患想在前，学生返校前需进行申请，得到批准后凭借西安市“一码通”绿码及其他相关报告予以返校；准备工作干在前，学校科学部署、全面联动，为学生返校复学保驾护航。保卫处严阵以待，把好安全第一关保卫部（处）工作人员提前设置好返校学生入校通道，按照流程一丝不苟地做好查验证件、核查西安市“一码通”、测量体温等准备工作。在校园安防指挥中心，工作人员通过“智安交大”安防平台关注各校门口的情况，第一时间调配力量进行疏导或者增援。在兴庆校区的各校门口，红毯、一米线、观察区和各类温馨提示牌设置齐全醒目，着装整齐的校卫队员们，一丝不苟地为返校学子核查证件、测量体温，并提醒同学们提前扫描西安市“一码通”，以便快速进入学校。在雁塔校区，保卫处工作人员设置了学生返校的专用通道，安排校卫队员在扫码处提醒返校同学们提前扫码。在创新港，校卫队员们认真做好门禁管理工作的同时，还充分利用空余时间和港区内巡逻用车辆，主动为到宿舍路程较远的同学提供行李运送服务，在做好安全管理的同时，为返校的同学们提供贴心的服务保障。后勤人温暖护航，提供坚实服务保障在雁塔校区，能源服务部对校园水电暖设施设备进行全面检修，全力做好开学前水电暖供应保障。根据天气动态调节供暖温度，对二次供水设施设备进行重点检查，按要求做好水质监测、消毒、化验等工作，确保师生饮用水安全。物业服务部对校园进行全面清扫保洁、消杀，对教室设施进行全面检修，确保教学工作顺利进行。在创新港：综合管理服务中心组织人员做好能源、物业、餐饮等各方面筹备工作，为师生们提供良好的后勤保障服务。保洁人员对教室、餐厅等公共区域进行打扫、消杀，为返校学生提供舒适的学习生活环境。在学生返校前，创新港后勤人员就在进行全面防疫消杀工作，保证学子安全入校。入校口、公寓口学生进行测温登记，筑牢防疫安全网。后勤保障部还安排小巴车帮学生搬运行李，服务工作暖心周到。为做好医疗保障，校医院提前谋划，与学工、研工等部门联合，制定《2021年春季学期学生返校开学疫情防控方案》《春季开学疫情防控应急处置预案》；加强预检分诊及医疗服务，门急诊24小时值班；做好防疫物资储备

In [26]:
# 导入停用词
stopwords = [line.strip() for line in open('./data/stop_words.txt',encoding='UTF-8').readlines()]
# print(stopwords)

In [27]:
# 使用jieba分词并去除停用词
seg_news_list = []
for news in news_list:
    seg_news = list(jieba.cut(news))
    # print('/'.join(seg_news))
    for word in seg_news:
        if word in stopwords:
            seg_news.remove(word)
    seg_news_list.append(' '.join(seg_news))
print(seg_news_list[0])

春到 人间 草木 知 西安交大 校园内 杨柳依依 梅花 浅笑 迎接 2021 春季 学期 返校 学子 教育部 陕西省 西安市 疫情 防控 要求 学校 有序 返校 原则 安排 国 境 学生 月 27 — 28 返校 注册 学校 部门 上阵 安全隐患 想 前 学生 返校 前 需 进行 申请 得到 批准 后 西安市 一码 通 绿码 他 相关 报告 予以 返校 准备 工作 干 前 学校 科学 部署 全面 联动 学生 返校 复学 保驾护航 保卫处 严阵以待 把 好 安全 第一关 保卫部 处 工作人员 提前 设置 好 返校 学生 入校 通道 按照 流程 一丝不苟 做好 查验 证件 核查 西安市 一码 通 测量体温 准备 工作 校园 安防 指挥 中心 工作人员 智安 交大 安防 平台 关注 各校 门口 情况 第一 时间 调配 力量 进行 疏导 增援 兴庆 校区 各校 门口 红毯 一 米线 观察 区 各类 温馨 提示牌 设置 齐全 醒目 着装 整齐 校卫 队员 一丝不苟 地为 返校 学子 核查 证件 测量体温 并 提醒 同学 提前 扫描 西安市 “ 一码 通 快速 进入 学校 雁塔 校区 保卫处 工作人员 设置 学生 返校 专用 通道 安排 校卫 队员 扫码 处 提醒 返校 同学 提前 扫码 在 创新 港 校卫 队员 认真 做好 门禁 管理工作 同时 充分利用 空余 时间 港区 巡逻 车辆 主动 宿舍 路程 较远 同学 提供 行李 运送 服务 在 做好 安全 管理 同时 为 返校 同学 提供 贴心 服务 保障 后勤 人 温暖 护航 提供 坚实 服务 保障 在 雁塔 校区 能源 服务部 校园 水电 暖 设施 设备 进行 全面 检修 全力 做好 开学 前 水电 暖 供应 保障 根据 天气 动态 调节 供暖 温度 二次 供水 设施 设备 进行 重点 检查 按 要求 做好 水质 监测 消毒 化验 工作 确保 师生 饮用水 安全 物业 服务部 校园 进行 全面 清扫 保洁 消杀 对 教室 设施 进行 全面 检修 确保 教学 工作 顺利进行 在 创新 港 综合 管理 服务中心 组织 人员 做好 能源 物业 餐饮 方面 筹备工作 为 师生 提供 良好 后勤保障 服务 保洁 人员 对 教室 餐厅 公共 区域 进行 打扫 消杀 为 返校 学生 提供 舒适 学习 生活 环境 在 学生 返校 前 创新 

In [28]:
vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000, min_df=2)
X = vectorizer.fit_transform(seg_news_list)
print("n_samples: %d, n_features: %d" % X.shape)

n_samples: 105, n_features: 4850


In [29]:
# 使用jieba自带的功能提取关键词和权值
for x, w in jieba.analyse.extract_tags(seg_news_list[0], topK = 20, withWeight = True):
    print('%s %s' % (x, w))

返校 0.46635947712083337
学生 0.2305312728768
防疫 0.09968852019250002
学子 0.09875325961239999
做好 0.0958701357894
书院 0.09417020706764999
开学 0.09080396086369999
学生宿舍 0.07969845001933333
西安市 0.07746256221658332
全面 0.07498704702066666
进行 0.0745424959004
同学 0.0720611559345
提前 0.0633799640525
工作 0.06304109013525
注册 0.06076107007341667
新学期 0.06054459091350001
校卫 0.059773837514499994
消杀 0.059773837514499994
入校 0.058517265373000005
温馨 0.058088612307600006


In [30]:
print("Performing dimensionality reduction using SVD......")
svd = TruncatedSVD(n_components = 70)
X = svd.fit_transform(X)

explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(
        int(explained_variance * 100)))

Performing dimensionality reduction using SVD......
Explained variance of the SVD step: 82%


In [31]:
print(X.shape)

(105, 70)


In [32]:
print("Do clustering......")
n_cluster = 5
km = KMeans(n_clusters=n_cluster, init='k-means++', max_iter=100, n_init=1)
print("Clustering sparse data with %s" % km)
km.fit(X)

Do clustering......
Clustering sparse data with KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=5, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=5, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [33]:
print("Top terms per cluster:")
original_space_centroids = svd.inverse_transform(km.cluster_centers_)
order_centroids = original_space_centroids.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(n_cluster):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Top terms per cluster:
Cluster 0: 党史 教育 学习 会议 党员 军号 历史 群众 习近平 古田
Cluster 1: 学生 同学 图书馆 返校 我们 开学 生活 防控 疫情 学子
Cluster 2: 文化 文明 学生 西迁 精神文明 育人 教育 书院 军事 思想
Cluster 3: 人群 dna 遗传 研究 东亚地区 考古 遗址 农业 基因组学 南岛
Cluster 4: 科技 合作 表示 团队 西安交大 科研 西部 交大 研究 关键词
